## Data Importing and Inspection

In [17]:
#import required libraries
import pandas as pd

#read the data
actual_generation = pd.read_csv('../data/raw/Actual_generation_202101010000_202509180000_Hour.csv', sep=';', low_memory=False)
forecasted_generation = pd.read_csv('../data/raw/Day-ahead_prices_202101010000_202509180000_Hour.csv', sep=';', low_memory=False)

In [18]:
#display the first few rows of the data
actual_generation.head()

,Start date,End date,Biomass [MWh] Calculated resolutions,Hydropower [MWh] Calculated resolutions,Wind offshore [MWh] Calculated resolutions,Wind onshore [MWh] Calculated resolutions,Photovoltaics [MWh] Calculated resolutions,Other renewable [MWh] Calculated resolutions,Nuclear [MWh] Calculated resolutions,Lignite [MWh] Calculated resolutions,Hard coal [MWh] Calculated resolutions,Fossil gas [MWh] Calculated resolutions,Hydro pumped storage [MWh] Calculated resolutions,Other conventional [MWh] Calculated resolutions
0,"Jan 1, 2021 12:00 AM","Jan 1, 2021 1:00 AM","4,481.00","1,203.00",383.00,"3,928.25",1.00,213.00,"8,144.75","11,608.50","3,443.75","6,923.00",347.25,"1,637.50"
1,"Jan 1, 2021 1:00 AM","Jan 1, 2021 2:00 AM","4,453.00","1,192.75",394.50,"3,528.25",1.00,213.25,"8,150.25","11,602.75","3,044.75","6,688.00",518.50,"1,636.00"
2,"Jan 1, 2021 2:00 AM","Jan 1, 2021 3:00 AM","4,440.50","1,161.00",305.25,"3,198.00",1.00,216.50,"8,156.50","11,758.50","3,067.25","6,586.00",170.00,"1,630.00"
3,"Jan 1, 2021 3:00 AM","Jan 1, 2021 4:00 AM","4,424.75","1,177.75",319.25,"2,768.75",1.00,218.00,"8,153.75","12,337.50","2,852.50","6,396.50",2.00,"1,621.50"
4,"Jan 1, 2021 4:00 AM","Jan 1, 2021 5:00 AM","4,430.75","1,153.25",296.25,"2,462.25",1.00,221.50,"8,150.50","12,395.00","2,713.25","6,333.00",6.00,"1,621.50"


In [15]:
forecasted_generation.head()


,Start date,End date,Germany/Luxembourg [€/MWh] Original resolutions,∅ DE/LU neighbours [€/MWh] Original resolutions,Belgium [€/MWh] Original resolutions,Denmark 1 [€/MWh] Original resolutions,Denmark 2 [€/MWh] Original resolutions,France [€/MWh] Original resolutions,Netherlands [€/MWh] Original resolutions,Norway 2 [€/MWh] Original resolutions,Austria [€/MWh] Original resolutions,Poland [€/MWh] Original resolutions,Sweden 4 [€/MWh] Original resolutions,Switzerland [€/MWh] Original resolutions,Czech Republic [€/MWh] Original resolutions,DE/AT/LU [€/MWh] Original resolutions,Northern Italy [€/MWh] Calculated resolutions,Slovenia [€/MWh] Original resolutions,Hungary [€/MWh] Original resolutions
0,"Jan 1, 2021 12:00 AM","Jan 1, 2021 1:00 AM",50.87,44.17,50.87,50.87,50.87,50.87,50.87,24.95,50.87,34.18,24.95,50.98,45.54,-,50.87,50.87,45.54
1,"Jan 1, 2021 1:00 AM","Jan 1, 2021 2:00 AM",48.19,41.63,48.19,48.19,48.19,48.19,48.19,24.35,48.19,32.61,24.35,45.94,41.59,-,48.19,48.19,41.59
2,"Jan 1, 2021 2:00 AM","Jan 1, 2021 3:00 AM",44.68,39.38,44.68,44.68,44.68,44.68,44.68,23.98,44.68,32.58,23.98,44.53,40.05,-,44.68,44.68,40.05
3,"Jan 1, 2021 3:00 AM","Jan 1, 2021 4:00 AM",42.92,37.73,42.92,42.92,42.92,42.92,42.92,23.72,42.92,32.20,23.72,41.01,36.90,-,42.92,42.92,36.90
4,"Jan 1, 2021 4:00 AM","Jan 1, 2021 5:00 AM",40.39,35.70,40.39,40.39,40.39,40.39,40.39,23.73,40.39,29.16,23.73,39.23,34.47,-,40.39,40.39,34.47


In [25]:
#check for missing values
print(actual_generation.isnull().values.any())
print(forecasted_generation.isnull().values.any())

False
False


In [22]:
#display data types
print(actual_generation.dtypes)
print(forecasted_generation.dtypes)

Start date                                           object
End date                                             object
Biomass [MWh] Calculated resolutions                 object
Hydropower [MWh] Calculated resolutions              object
Wind offshore [MWh] Calculated resolutions           object
Wind onshore [MWh] Calculated resolutions            object
Photovoltaics [MWh] Calculated resolutions           object
Other renewable [MWh] Calculated resolutions         object
Nuclear [MWh] Calculated resolutions                 object
Lignite [MWh] Calculated resolutions                 object
Hard coal [MWh] Calculated resolutions               object
Fossil gas [MWh] Calculated resolutions              object
Hydro pumped storage [MWh] Calculated resolutions    object
Other conventional [MWh] Calculated resolutions      object
dtype: object
Start date                                          object
End date                                            object
Germany/Luxembourg [€/MWh] O

## Data Cleaning

In [ ]:
# Turn the 'Start date' and "End date" columns into a datetime object
date_format = "%b %d, %Y %I:%M %p"

actual_generation['Start date'] = pd.to_datetime(actual_generation['Start date'], format=date_format)
actual_generation['End date'] = pd.to_datetime(actual_generation['End date'], format=date_format)
forecasted_generation['Start date'] = pd.to_datetime(forecasted_generation['Start date'], format=date_format)
forecasted_generation['End date'] = pd.to_datetime(forecasted_generation['End date'], format=date_format)

In [29]:
# Turn all the object columns into float
for df in [actual_generation, forecasted_generation]:
    for col in df.select_dtypes(include=['object']).columns:
        # Remove commas and convert to float, ignore errors for non-numeric columns
        df[col] = df[col].str.replace(',', '').astype(float, errors='ignore')